In [1]:
"""
Deep Deterministic Policy Gradient (DDPG), Reinforcement Learning.
P2P, net bit rate, energy harvesting example for validation.
Using:
tensorflow 1.0
"""
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import gym
import time
import EH_P2P
import math

MAX_EPISODES = 1
MAX_EP_STEPS = 100000

RENDER = False
OUTPUT_GRAPH = True

env=EH_P2P.EH_P2P()
env.Chanpower()
env.Solarread()
B=np.zeros((100,4))    

def dense(x,a,b):
    results=(x.dot(a)+b)
    return results

def choose_action(s):
    a=dense(s,a1,b1)
    a=dense(a,a3,b3)
    a=dense(a,aa,ba)
    a=1/(1+np.exp(-a))
    a = np.clip(a, 0, 1)
    return a

for snr in range (-10,-8,2):

    for epoch in range (0,1750,30):

        modulation=0

        tf.reset_default_graph()
        graph = tf.get_default_graph()
        
        saver = tf.train.import_meta_graph("folder_for_nn_noise"+"/EH_save_net_snr="+str(snr)+str(modulation)+"epoch="+str(epoch)+"_P2P.ckpt.meta")
        with tf.Session() as sess:
            saver.restore(sess,"folder_for_nn_noise"+"/EH_save_net_snr="+str(snr)+str(modulation)+"epoch="+str(epoch)+"_P2P.ckpt")

            a1 = sess.run('Actor/eval_net/l1/kernel:0')
            b1 = sess.run('Actor/eval_net/l1/bias:0')
            a3 = sess.run('Actor/eval_net/l3/kernel:0')
            b3 = sess.run('Actor/eval_net/l3/bias:0')
            aa = sess.run('Actor/eval_net/a/a/kernel:0')
            ba = sess.run('Actor/eval_net/a/a/bias:0')
    

    
            for i in range(MAX_EPISODES):

                s = env.reset_P2P(snr=snr)
                s=np.reshape(s,(1,-1))
                ep_reward = 0
                for j in range(MAX_EP_STEPS):
                    s=np.array(s,dtype=float)
                    a=choose_action(s)
                    s_, r, info = env.step_P2P([a,modulation])
                    s = s_
                    ep_reward += r
                    if (j+1)%10000==0:
                        print("net bit rate=",ep_reward/j,"snr=",snr,"modulation=",modulation, "loop =",i,"action=",a,"noise=",env.noise)
          
                index=(snr+10)/2
                B[int(index),int(modulation)]=ep_reward/j
                print(B[int(index),int(modulation)])



Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from folder_for_nn_noise/EH_save_net_snr=-100epoch=1999_P2P.ckpt


I0000 00:00:1732833756.139955  243944 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
/opt/anaconda3/lib/python3.12/site-packages/tensorflow/python/client/session.py:1483: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return tf_session.TF_SessionRun_wrapper(self._session, options, feed_dict,


ValueError: Argument `fetch` = Actor/eval_net/a/a/kernel:0 cannot be interpreted as a Tensor. ("The name 'Actor/eval_net/a/a/kernel:0' refers to a Tensor which does not exist. The operation, 'Actor/eval_net/a/a/kernel', does not exist in the graph.")

In [1]:
"""
Deep Deterministic Policy Gradient (DDPG), Reinforcement Learning.
P2P, net bit rate, energy harvesting example for validation.
Using:
tensorflow 1.0
"""
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import gym
import time
import EH_P2P
import math

MAX_EPISODES = 1
MAX_EP_STEPS = 100000

RENDER = False
OUTPUT_GRAPH = True

env=EH_P2P.EH_P2P()
env.Chanpower()
env.Solarread()
B=np.zeros((100,4))    

def dense(x,a,b):
    results=(x.dot(a)+b)
    return results

def choose_action(s):
    # Fix input shape mismatch if necessary
    if s.shape[1] != a1.shape[0]:
        #print(f"Padding input s from shape {s.shape} to match a1 shape {a1.shape}")
        s = np.pad(s, ((0, 0), (0, a1.shape[0] - s.shape[1])), 'constant')
    
    # if a1.shape[1] != b1.shape[0]:
    #     #print(f"Padding input a1 from shape {a1.shape} to match b1 shape {b1.shape}")
    #     a1 = np.pad(s, ((0, 0), (0, b1.shape[0] - a1.shape[1])), 'constant')
    
    # if a1.shape[1] != b1.shape[0]:
    #     #print(f"Padding input a1 from shape {a1.shape} to match b1 shape {b1.shape}")
    #     a1 = np.pad(s, ((0, 0), (0, b1.shape[0] - a1.shape[1])), 'constant')

    print("Reshaped s: ", s.shape)
    print("Reshaped a1: ", a1.shape)
    
    a=dense(s,a1,b1)
    a=dense(a,a3,b3)
    a=dense(a,aa,ba)

    # a=dense(s, a1_reshaped, b1_reshaped)
    # a=dense(s, a3_reshaped, b3_reshaped)
    # a=dense(s, aa_reshaped, ba_reshaped)

    a=1/(1+np.exp(-a))
    a = np.clip(a, 0, 1)
    return a


snr=-10
epoch=1999
modulation=0

tf.reset_default_graph()
graph = tf.get_default_graph()

saver = tf.train.import_meta_graph("folder_for_nn_noise"+"/EH_save_net_snr="+str(snr)+str(modulation)+"epoch="+str(epoch)+"_P2P.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess,"folder_for_nn_noise"+"/EH_save_net_snr="+str(snr)+str(modulation)+"epoch="+str(epoch)+"_P2P.ckpt")

    a1 = sess.run('Actor/eval_net/l1/kernel:0')
    b1 = sess.run('Actor/eval_net/l1/bias:0')
    a3 = sess.run('Actor/eval_net/l3/kernel:0')
    b3 = sess.run('Actor/eval_net/l3/bias:0')
    aa = sess.run('Actor/eval_net/a/kernel:0')
    ba = sess.run('Actor/eval_net/a/bias:0')

    print(f"Shape a1: {a1.shape}, Shape b1: {b1.shape}")
    print(f"Shape a3: {a3.shape}, Shape b3: {b3.shape}")
    print(f"Shape aa: {aa.shape}, Shape ba: {ba.shape}")
    
    # input_dim = 168  # Adjust this to match your model's expected input size
    # output_dim = 168  # Adjust this to match your model's hidden layer size

    # a1 = np.reshape(a1, (input_dim, output_dim))
    # b1 = np.reshape(b1, (output_dim,))
    # a3 = np.reshape(a3, (output_dim, output_dim))
    # b3 = np.reshape(b3, (output_dim,))
    # aa = np.reshape(aa, (output_dim, output_dim))
    # ba = np.reshape(ba, (output_dim,))

    # print(f"Reshaped a1: {a1.shape}, Reshaped b1: {b1.shape}")
    # print(f"Reshaped a3: {a3.shape}, Reshaped b3: {b3.shape}")
    # print(f"Reshaped aa: {aa.shape}, Reshaped ba: {ba.shape}")

    for i in range(MAX_EPISODES):

        s = env.reset_P2P(snr=snr)
        s=np.reshape(s,(1,-1))
        ep_reward = 0
        for j in range(MAX_EP_STEPS):
            s=np.array(s,dtype=float)
            a=choose_action(s)
            s_, r, info = env.step_P2P([a,modulation])
            s = s_
            ep_reward += r
            if (j+1)%10000==0:
                print("net bit rate=",ep_reward/j,"snr=",snr,"modulation=",modulation, "loop =",i,"action=",a,"noise=",env.noise)
    
        index=(snr+10)/2
        B[int(index),int(modulation)]=ep_reward/j
        print(B[int(index),int(modulation)])

Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from folder_for_nn_noise/EH_save_net_snr=-100epoch=1999_P2P.ckpt
Shape a1: (168,), Shape b1: (168,)
Shape a3: (168,), Shape b3: (168,)
Shape aa: (168,), Shape ba: (168,)


I0000 00:00:1732848049.265537  361107 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
/opt/anaconda3/lib/python3.12/site-packages/tensorflow/python/client/session.py:1483: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return tf_session.TF_SessionRun_wrapper(self._session, options, feed_dict,


UnboundLocalError: cannot access local variable 'a1' where it is not associated with a value

In [ ]:
print(B)

In [ ]:
np.savetxt("DDPG_P2P_noise2"+str(snr)+".csv", B, delimiter = ',')